In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
file_path = "./resume.pdf"

In [5]:
loader = PyPDFLoader(file_path)
document = loader.load()

Ignoring wrong pointing object 134 0 (offset 0)


In [6]:
document[0].page_content[:100]

'Shashank  Kumar  Rajak  Raipur,  India  |  +91-7509616100  |  shashanksrajak@gmail.com https://shash'

In [45]:
len(document[0].page_content)

6130

## Embeddings
converting sentences into vectors

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [7]:
query = embeddings.embed_query("Hello, world!")
len(query)

768

## Text Splitter

In [66]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter


text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(document)

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust as needed
    chunk_overlap=50,  # Helps maintain context across chunks
    separators=["\n\n", "\n", " ", ""],  # Prioritized list of separators
)

split_docs = recursive_text_splitter.split_documents(document)

In [67]:
# len(split_docs)
split_docs[0]

Document(metadata={'producer': 'Skia/PDF m137 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Shashank_Rajak_ML_Engineer_Resume', 'source': './resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="Shashank  Kumar  Rajak  Raipur,  India  |  +91-7509616100  |  shashanksrajak@gmail.com https://shashankrajak.in |  Github  |   LinkedIn  |   Kaggle  WORK  EXPERIENCE  Lead  Backend  Engineer  |  Binaryveda  Software  Solutions  Pvt  Ltd,  Mumbai |  Sep  2021  –  August  2024  (continued  as  part-time  consultant:  October  2024  –  January  2025 )  -  Led  a  backend  engineering  team  of  5-6,  delivering  multiple  projects  across  industries  (IoT,  EV,  SaaS,  B2B/B2C)  using  \nNode.js,\n \nNestJS,\n \nExpress,\n \nand\n \nAWS\n \ncloud\n \nservices.\n \nArchitected\n \nscalable\n \ncloud\n \narchitectures,\n \nCI/CD\n \npipelines,\n \nand\n \nimplemented\n \nperformance\n \ntuning\n \nstrategies.\n -  Automated  Kodak's competitive  sear

In [48]:
len(docs)

1

## Vector Datastore

In [31]:
from langchain_deeplake.vectorstores import DeeplakeVectorStore

In [72]:

activeloop_org_id = "shashanksrajak"
activeloop_dataset_name = "langchain_tutorial_resume_personal"
dataset_path = f"hub://{activeloop_org_id}/{activeloop_dataset_name}"

In [73]:
db = DeeplakeVectorStore(dataset_path=dataset_path, embedding_function=embeddings)

In [75]:
db.add_documents(split_docs)

['c593c780-65e7-458a-a267-60163f2c5fa2',
 'f54baf42-b807-458e-b102-3c9d31804ca1',
 '90053b0c-fc88-42ed-a1c8-42001a179f0d',
 'a9744559-9d56-4448-812e-3032082fe840',
 'ac4997c7-4601-4216-9600-36484dac193d',
 '854ca565-145c-4715-9f70-35caf002acdb',
 'fede8023-c03e-4fa7-b17e-6c2fa4c5bc7b']

In [78]:
query = "What is his education?"

In [81]:
similar_docs = db.similarity_search(query)
for doc in similar_docs:
    print(doc.page_content)
    print("-------------------------")

Shashank  Kumar  Rajak  Raipur,  India  |  +91-7509616100  |  shashanksrajak@gmail.com https://shashankrajak.in |  Github  |   LinkedIn  |   Kaggle  WORK  EXPERIENCE  Lead  Backend  Engineer  |  Binaryveda  Software  Solutions  Pvt  Ltd,  Mumbai |  Sep  2021  –  August  2024  (continued  as  part-time  consultant:  October  2024  –  January  2025 )  -  Led  a  backend  engineering  team  of  5-6,  delivering  multiple  projects  across  industries  (IoT,  EV,  SaaS,  B2B/B2C)  using  
Node.js,
 
NestJS,
 
Express,
 
and
 
AWS
 
cloud
 
services.
 
Architected
 
scalable
 
cloud
 
architectures,
 
CI/CD
 
pipelines,
 
and
 
implemented
 
performance
 
tuning
 
strategies.
 -  Automated  Kodak's competitive  search  engine  analysis,  replacing  a  manual  process.  Developed  a  system  that  performs  web  scraping  of  SERPs  from  Google,  Baidu,  and  Naver  (for  ~10  countries),  calculates  keyword  metrics  for  
Kodak
 
and
 
its
 
competitors,
 
and
 
automatically
 
updates
-

In [82]:
from langchain.chat_models import init_chat_model

In [84]:
model = init_chat_model(model="gpt-3.5-turbo",
                        model_provider="openai", temperature=0)

In [85]:
model.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BWKVmQnjgmhh9AgVLm1UiU2H9tj7I', 'finish_reason': 'stop', 'logprobs': None}, id='run-8661c13b-1ae5-4ba4-a083-3c8eb6061830-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [86]:
template = """
Your are an exceptional AI assistant helping recruiters analyze a candidate's suitability for the role of AI Engineer.

You know the following context information.

{chunks_formatted}

Answer to the following question from a recruiter. Use only information from the previous context about the candidate. Do not invent stuff.

Question: {query}

Answer:
"""

In [87]:
from langchain_core.prompts import ChatPromptTemplate


prompt_template = ChatPromptTemplate.from_template(template)

In [90]:
prompt_formatted =  prompt_template.format(chunks_formatted="IIT Kharagpur, Civil Engineering, IIT Guwahati, Data Science & AI, BSc", query="what is his educational background?")

In [91]:
prompt_formatted

"Human: \nYour are an exceptional AI assistant helping recruiters analyze a candidate's suitability for the role of AI Engineer.\n\nYou know the following context information.\n\nIIT Kharagpur, Civil Engineering, IIT Guwahati, Data Science & AI, BSc\n\nAnswer to the following question from a recruiter. Use only information from the previous context about the candidate. Do not invent stuff.\n\nQuestion: what is his educational background?\n\nAnswer:\n"

In [ ]:
response = model.invoke(prompt_formatted)
response.content

AIMessage(content='The candidate has a background in Civil Engineering from IIT Kharagpur and has further pursued studies in Data Science & AI at IIT Guwahati. Additionally, they hold a BSc degree.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 97, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BWKenld0vMU4sfzTQvdqE8n7zGbwM', 'finish_reason': 'stop', 'logprobs': None}, id='run-da628c16-a5ce-4cfe-9146-b0926247c9bc-0', usage_metadata={'input_tokens': 97, 'output_tokens': 43, 'total_tokens': 140, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [93]:
response.content

'The candidate has a background in Civil Engineering from IIT Kharagpur and has further pursued studies in Data Science & AI at IIT Guwahati. Additionally, they hold a BSc degree.'

## Final QnA

In [97]:
query = "What skills he has in AI & Machine Learning?"

In [98]:
docs = db.similarity_search(query)

retrieved_chunks = [doc.page_content for doc in docs]

chunks_formatted = "\n\n".join(retrieved_chunks)

prompt_formatted = prompt_template.format(chunks_formatted=chunks_formatted, query=query)

# generate answer
response = model.invoke(prompt_formatted)

print(response.content)

Shashank Kumar Rajak has skills in AI and Machine Learning, including programming languages such as Python and JavaScript, as well as libraries like TensorFlow, Keras, and Scikit-learn. He also has experience with NLP (Natural Language Processing) and holds a certification as an AWS Certified Solutions Architect - Associate.


In [96]:
response

AIMessage(content="Yes, based on the candidate's work experience as a Lead Backend Engineer at Binaryveda Software Solutions Pvt Ltd, where they led a team delivering projects using Node.js, NestJS, Express, and AWS cloud services, as well as their education in Data Science & Artificial Intelligence, certifications in AWS, and skills in Python, JavaScript, and NLP, it is evident that this candidate is highly suitable for the role of AI Engineer. Their experience in developing AI chatbots, working on AI projects like Kodak's competitive search engine analysis, and leading the backend development of projects like Simple Energy and smart lock systems for Godrej Locks showcases their expertise in AI technologies and their ability to architect scalable cloud architectures and implement performance tuning strategies. Their background in Civil Engineering also demonstrates their ability to adapt and learn new technologies, making them a strong candidate for the role.", additional_kwargs={'ref